# Fluxcourse SIF section

## Introduction
In this section, we will learn how to process SIF data from multiple satellite platforms. At the end of this section, we will compare satellite SIF data with GPP estimation from a few FLUXNET towers. We will go through the following sections:

* The start: how to read SIF data?
* Post-processing: how to create scientifically useful datasets and visualize them?
* I want to create fancy maps, how to do it?
* Do what we see from the satellite agree with what we see from the ground (i.e. FLUXNET or Ameriflux)?

## Logistics
We use Jupyter Notebook for this practice. Remember to install the packages using "conda install [package-name]" in your command line. Before the course, remember to update your conda (if you use anaconda) by "conda update conda".
You will need the following packages:

* matplotlib
* numpy
* scipy
* h5py
* netCDF4
* glob
* traceback
* geoviews
* xarray

A large portion of this code is from **Christian Frankenberg and Philipp Koehler**. L2_tools.py provides the tools to use TROPOMI and OCO-2 L2 data. You will need to have the following files under the same folder of this code:

* L2_tools.py: read SIF data; calculate running mean
* spectral_shape.dat: the spectral shape you created from the previous section

Another useful tool to read netCDF file is Panoply. It is a free software from NASA. Download it from here: https://www.giss.nasa.gov/tools/panoply/

In [ ]:
# Remember to install these packages
import matplotlib.pyplot as plt
import numpy as np
import scipy
# nicer figures using ggg plot style.
plt.style.use('ggplot')
from IPython.core.display import HTML
from IPython.display import clear_output
from IPython.core.pylabtools import figsize

# hide all warnings
import warnings
warnings.filterwarnings('ignore')

**Remember to add L2_tool.py to the same folder**

In [ ]:
# This file L2_tools.py is in the same folder here and will provide easy access to TROPOMI and OCO-2 L2 data 
from L2_tools import *

In [ ]:
# Use your local directory
path_tropomi = '/Volumes/data2/projects/TROPOMI/nc_ungridded/'
path_oco2 = '/Volumes/data1/ftp/data/OCO2/sif_lite_B8100/'

# This is still cumbersome, the units are part of the netCDF4 files but I hardcode them for now:
t_unit_oco2    = "seconds since 1993-1-1 0:0:0"
t_unit_tropomi = "seconds since 1970-01-01 00:00:00"
t_cal = u"gregorian"

In [ ]:
import pandas as pd

path = '/Volumes/XiYangBackUp/Data/33.FluxCourse/Tonzi_DailyGEE2.csv'
df   = pd.read_csv(path, skiprows=1)